In [1]:
from elasticsearch import Elasticsearch
from functools import partial

# Create an elastic search engine
es = Elasticsearch(
     cloud_id="lm-datasets:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDk1N2U5ODIwZDUxNTQ0YWViMjk0MmQwNzI1NjE0OTQ2JDhkN2M0OWMyZDEzMTRiNmM4NDNhNGEwN2U4NDE5NjRl",
     api_key="T2w5Vl9ZTUIzVzE5dTlBblUtRlo6MHNBYWxhbHVTeGFodUpUOWIybkNNZw==",
     retry_on_timeout=True,
     http_compress=True,
)

# Syntactic sugar to avoid repeating index and keyword arguments
search = partial(
    es.search,
    
    index="re_pile",
    track_total_hits=True,
)

In [2]:
docs = []

def process_hits(hits: list):
    global docs
    
    for doc in hits:
        docs.append(doc)
        
query = {
    "match": {"text": {"query": "Catarina Belem", "operator": "and"}}
}

data = search(query=query, size=100, scroll="10m")
print("Total documents found", data["hits"]["total"])

sid = data["_scroll_id"]
scroll_size = len(data["hits"]["hits"])

while scroll_size > 0:
    print("Processing", scroll_size, "documents")

    process_hits(data["hits"]["hits"])
    print(len(docs))

    data = es.scroll(scroll_id=sid, scroll="2m")
    sid = data["_scroll_id"]
    
    scroll_size = len(data["hits"]["hits"])

es.clear_scroll(scroll_id=sid)

Total documents found {'value': 291, 'relation': 'eq'}
Processing 100 documents
100
Processing 100 documents
200
Processing 91 documents
291


ObjectApiResponse({'succeeded': True, 'num_freed': 20})

In [4]:
docs[0]

{'_index': 're_pile',
 '_id': 'train/25.jsonl.gz-851667',
 '_score': 32.746773,
 '_source': {'text': 'The Belem curassow\n\nThe Belem curassow ( Crax pinima ) was recently split off from the Bare-faced curassow and with this became directly the rarest bird-species !\nIn the wild it is most prop. extinct and in captivity 5 birds are known.\nA male and 2 females are kept at a breeding station in Santa Catarina and 2 females are kept at a breeding centre in Minas Gerais ( information IUCN Redlist 2013 ).\nNow I want to ask if anybody knows more about these 2 breeding-stations and if in the meantime the species already has been bred ?',
  'meta': {'pile_set_name': 'Pile-CC'}}}